In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import math
import torch
import sys
import json
import re
from tqdm import tqdm
from llama_attn_replace import replace_llama_attn
import deepspeed
import gc

[2023-12-02 20:54:34,359] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## DeepSpeed Offloading

In [2]:
ds_config = {
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": True
        }
    }, 
    "train_batch_size": 1,
    'weight_quantization': {
        'quantized_initialization' : {
            'num_bits': 4,
            'group_size': 64,
            "group_dim": 1,
            "symmetric": False
        }
    },
}

## Parameters

In [3]:
PROMPT_DICT = {
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
    "prompt_no_input_llama2": (
        "<s>[INST] <<SYS>>\n"
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"
        "<</SYS>> \n\n {instruction} [/INST]"
    ),
    "prompt_llama2": "[INST]{instruction}[/INST]"
}

In [4]:
model_name = "Yukang/LongAlpaca-7B"
cache_dir = "../../cache"
context_size = 32768

## Config, Model and Tokenizer

In [5]:
replace_llama_attn(inference=True)

In [6]:
config = AutoConfig.from_pretrained(model_name, cache_dir=cache_dir)

In [7]:
orig_ctx_len = getattr(config, "max_position_embeddings", None)
if orig_ctx_len and context_size > orig_ctx_len:
    scaling_factor = float(math.ceil(context_size / orig_ctx_len))
    config.rope_scaling = {"type": "linear", "factor": scaling_factor}

In [8]:
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        config=config,
        cache_dir=cache_dir,
        torch_dtype=torch.float16,
        device_map="auto",
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/pytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /opt/conda/envs/pytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/opt/conda/envs/pytorch/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda/envs/pytorch did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model.resize_token_embeddings(32001)

Embedding(32001, 4096, padding_idx=0)

In [10]:
model_engine, _, _, _ = deepspeed.initialize(config_params=ds_config, model=model)

[2023-12-02 20:55:12,197] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.12.4, git-hash=unknown, git-branch=unknown
[2023-12-02 20:55:12,198] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-12-02 20:55:12,198] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-12-02 20:55:12,320] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.31.18.216, master_port=29500
[2023-12-02 20:55:12,320] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-12-02 20:55:13,159] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2023-12-02 20:55:13,161] [INFO] [logging.py:96:log_dist] [Rank 0] Creating ZeRO Offload
[2023-12-02 20:55:13,269] [INFO] [utils.py:795:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2023-12-02 20:55:13,270] [INFO] [utils.py:796

In [11]:
model_engine = model_engine.to("cuda")

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        model_max_length=context_size if context_size > orig_ctx_len else orig_ctx_len,
        padding_side="right",
        use_fast=False,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Inference: MB

In [13]:
def build_generator(
    model_engine, tokenizer, temperature=0.6, top_p=0.9, max_gen_len=512, use_cache=True
):
    def response(prompt):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        #streamer = TextStreamer(tokenizer)
        
        output = model_engine.module.generate(
            **inputs,
            max_new_tokens=max_gen_len,
            temperature=temperature,
            top_p=top_p,
            use_cache=use_cache,
            #streamer=streamer,
        )
        
        out = tokenizer.decode(output[0], skip_special_tokens=True)

        out = out.split(prompt.lstrip("<s>"))[1].strip()
        return out

    return response

In [14]:
model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

respond = build_generator(model_engine, tokenizer, max_gen_len=512)

In [15]:
prompt_no_input = PROMPT_DICT["prompt_llama2"]

In [16]:
question = "Summarize the meeting transcrip."

### Single Example

In [17]:
material = "Item 2017-0475 transcripts:\nspeaker 0: Councilmember Gossett So this is an opportunity to speak to it. Okay, very good. All right. Thank you very much. Staff. We will move on to proposed ordinance number 2017 0475. This item would approve and adopt two memoranda of agreement with the Joint Labor Management Insurance Committee, which represents benefits eligible employees represented by labor organizations except transit operators\nspeaker 0: and sheriff's deputies. Understand, we have an updated fiscal note that will be passed around, hopefully from what was included in the packet. Is that correct? That is correct. Thank you. And here to welcome our guests and give us a briefing is council staff Heidi.\nspeaker 0: Papa Chuck. Thank you. Thank you, Madam Chair. How do you provide our council staff? And also to my right is Bob Railton of the King County Office of Labor Relations. To his right is Denise Cobden from Professional and Technical Employees Local 17 Union.\nspeaker 0: And to her right is Kari Schaefer from the King County's Human Resources Division. The staff report begins on page 99 of your packet. Madam Chair, as you mentioned, proposed ordinance 2017 0475 would approve to joint labor management insurance.\nspeaker 0: Or JL am I c memorandum of Agreement or. Emojis. The first emoji would adopt an accountable health network or a H, and there's going to be a lot of acronyms in this presentation benefit plan designed for 2018 and 2019.\nspeaker 0: The second emoji would adopt the Health Maintenance Organization or HMO and preferred provider organization or PPO plan designs for 2019, and replace the Health Incentives Program with a health and well-being program tomorrow. To provide some background on the J.L. and Missy.\nspeaker 0: It consists of labor and county management representatives that collectively bargain ensure benefits for county employees except those that are part of the Amalgamated Transit Union, the Kane County Police Officers Guild, the Puget Sound Police Managers Association representing captains and the Technical Employees Association representing employees in the transit division.\nspeaker 0: They share benefits in most union contracts and for now represented employees are negotiated by this committee. In November 2016, Council adopted Ordinance 18405, which approved the employee regarding insured benefits. And this meeting I recall that the benefits agreement between king county and the I c unions that prescribed medical, dental vision and life insurance benefits for del amici\nspeaker 0: eligible employees for 2017 and 2018. There are three benefit agreement conditions described on page 100 of your packet that relates to proposed ordinance 2017 0475. The first condition describes the health and welfare plans the insured benefits plan provisions for JL Mis-C eligible employees during the term of the benefits agreement are described in tables one and two on\nspeaker 0: pages 100 and 101. The plan provisions in these two tables are tied to the county's Healthy Incentives program, which requires county employees to complete a wellness assessment and an individual action plan such as like Weightwatchers or to work out for six weeks prior to July 31st each year in order to achieve the goal level of the benefit\nspeaker 0: plan for the following year. The goal level benefit plan has the lowest out-of-pocket costs to achieve the silver or silver level benefit plan. Employees are required to complete the wellness assessment or the Individual Action Plan prior to July 31st each year.\nspeaker 0: If an employee does not complete the wellness assessment in the Individual Action Plan by the deadline, they would maintain the bronze level benefit plan, which has the highest out-of-pocket costs. All county employees start at the bronze level benefit plan.\nspeaker 0: The employee's spouse or domestic partner would also have to complete the same requirements in order to achieve the gold, silver or bronze level benefit plan. The second condition describes the G, the J.L. and Mike's ability to negotiate and implement modifications to ensure benefits for JL am i c eligible employees during the term of the benefits agreement?\nspeaker 0: The third condition describes that the deal and MRE will add supplemental plan options beyond the HMO plan and that PPO plan for the 2018 benefit year in 2017. Council approved motion 14877, which acknowledged the receipt of the executive's budget proviso response report on accountable health networks or in the report can be found on page 119 of your\nspeaker 0: packet. In addition, the third condition describes that the JL and Massie agrees to negotiate changes to the Healthy Incentives Program to be effective for the 2018 benefit year. In accordance with the Benefits Agreement, the first proposed J.L. and Missy Millard would adopt an eight, eight and a plan for the 2018 benefit year, according to executive staff.\nspeaker 0: And an 18 is a form of value based purchasing. It brings together physicians, hospitals and other partners into focus networks for the amount of money the network receives for treating a population of patients is based in part on the quality of care that they deliver and the patient satisfaction satisfaction with the care they experience.\nspeaker 0: Executive staff offer the age and plan in the open enrollment period for benefits in 2018. The proposed ordering, which was transmitted to the Council on November 1st, 2017, would approve the HRA and plan designs for 2018 and 2019.\nspeaker 0: Table three on page 103 illustrates the 2018 and 2019 King Cares. Select a H and plan design that was agreed to by the JL and Missy. According to Executive staff, eligible employees will be able to select one of the four eight.\nspeaker 0: Available in the Puget Sound region, Eastside Health Network, the Everett Clinic, Multi Care Connected Care and University of Washington Medicine. The proposed ordinance would also authorize the following provisions as an incentive to the select HMO plan option. The annual deductible for 2018 and 2019 will be $0 for per single, only with a maximum of $400 per family\nspeaker 0: . Beginning in 2020, the annual deductible rates will be $200 per single, only with a maximum of $600 per family. And also in the event that at least 15% of eligible employees elect the plan option during the fall 2018 open enrollment.\nspeaker 0: The LMC will negotiate additional short term and or long term disability plans to be implemented no earlier than 2020. Should the JL and Missy reach an agreement on one end or both? Disability plan options? The county did not offer an HMO plan benefit plan option for its employees in the past.\nspeaker 0: Attachment three on page 135 of your packet illustrates the 2018 benefit plan designs offer to eligible employees, which includes the term plan design, according to executive staff over 500 JL and may see eligible employees enrolled in the new Plan Kane Care Select for the 2018 benefit year.\nspeaker 0: It was assumed by the county's benefit benefits consultant Mercer, that the shift in enrollment from Kane Care Taking Care Select will be approximately 5%, which equated to $110,000 in cost savings for 2018 and 2019 combined, according to the executive's fiscal new executive staff state that the and Missy achieved a 6% shift enrollment for 2018 cost savings could\nspeaker 0: reach as high as 1.8 million if 25% of employees move from Comcare to Comcare select, which is the HMO plan. Executive staff also indicate that there were no Smart Care Connect, which is the HMO enrollees that shifted to Comcare select during the open enrollment period.\nspeaker 0: Also in accordance with the JL and Missy Benefits Agreement, the second proposed JL and Missy Molly would replace the Healthy Incentives Program with a health and well-being program. The proposed ordinance would also adopt the 2019 benefit plan designs for the HMO in the PPO plans to reflect the alarm to reflect the elimination of the Healthy Incentives Program\nspeaker 0: . According to Executive staff, under the Health and Well-Being Program, starting in 2018, eligible employees and their spouse or domestic partner will no longer be required to complete the process steps of the Healthy Incentives Program to determine their benefit color level.\nspeaker 3: Can you can you say that again, as I think that's some new magic there?\nspeaker 0: Yeah. Yeah. So under the Health and Wellbeing Health and Wellbeing program, starting in 2018, eligible employees and their spouse or domestic partner will no longer be required to complete the process steps of the Healthy Incentives Program that in order to determine your benefit level by color, the gold, silver and bronze.\nspeaker 3: That's a below forms with all of the tax, all of that, the business about how we're climbing stairs and running blocks, correct?\nspeaker 2: You mean the way the ridiculously arduous and paperwork and necessary paperwork that everyone that's an employee can create, as I was referring to.\nspeaker 0: So that is correct.\nspeaker 3: All right. That was to the negotiators on that.\nspeaker 0: Thank you to the gentleman.\nspeaker 3: So we're going to tell us how it is determined.\nspeaker 0: Yeah, okay. Okay, good. Can I ask people to please ask to be recognized? Thank you.\nspeaker 2: We're just so excited about that. And then chair, I.\nspeaker 0: I try to bring it in written. Thank you. So table for on page 104 of your packet illustrates the proposed plan designs for 2019. The plan designs are the same as the goal level plan designs for the HMO and PPO plans and 2018 shown in Attachment three on page 135 of your packet.\nspeaker 0: According to executive staff, no significant cost savings are expected to be realized due to the discontinuation of the Healthy Incentives Program. Executive staff also indicate that the cost savings will be utilized to implement the Health and Well-Being Program.\nspeaker 0: Council staff requested additional information on the count, the costs associated with the shift of employees to the goal plan who would have been in the silver and bronze plans and 2019 executive staff has provided council staff with a revised fiscal note, which is on the dice.\nspeaker 0: I believe you have that to reflect this impact. The costs would increase approximately $600,000 to implement this proposed deal, and I am aware that concludes my staff report. Madam Chair, we are happy. To answer questions at the start.\nspeaker 0: There will be some questions. Councilmember Gossett.\nspeaker 1: Thank you, Madam Chair. Is there any way that all 13,000 of our staff are going to learn about or have already begun to learn about all this alphabet soup you just shared with us? And will there be anybody that I can check with to see if they have a sense that the staff understood all this?\nspeaker 1: So the union's who?\nspeaker 0: So as far as what they eat and benefit plan that was offered in the open enrollment process for 2017 and offered it will be available for the 2018 benefit year, in my understanding. Executive staff did provide some outreach and marketing materials for employees to kind of advise them of the new the additional plan.\nspeaker 0: And as far as the 2019 changes for the healthy incentives, I will defer to executive staff on to to discuss the kind of outreach and educational steps to inform employees.\nspeaker 1: Okay. So I'm going to offer an opinion and have before I have one more question for her.\nspeaker 2: And Councilmember Gossett. Bob Railton, Labor Relations Manager. Office of Labor Relations. Kerry Schaefer actually is very much part of the communication process where we educate employees about the various planning choices that's available to them coming into open enrollment, as well as when employees first begin their employment with King County and they have an opportunity to select the\nspeaker 2: plans. We do have a very, very extensive outreach and communication plan designed to help educate employees, both in terms of the plan options that are available to them, but the various sort of decisions that they'll need to consider as they determine which plan option is best for them and their family.\nspeaker 2: Any more additional information term specifics? I'd have to turn to Carrie Schaefer.\nspeaker 1: Carrie, do you think that we have a system where staff will generally become relatively educated on this new system or at least read health care reforms?\nspeaker 0: Yes, Councilman.\nspeaker 1: Again.\nspeaker 0: I don't know if it's on. Okay. It's okay. Yes. Councilmember Garza, we have actually been educating employees over the past year. And you are hearing everything all at once. They've been hearing the story as it unrolls. So before open enrollment, we explained to employees what the new accountable health care, the King Care Select option was and what\nspeaker 0: kinds of how they could make a decision whether or not that was a good plan for them, because it's not the best plan for everybody. It's a it's a good plan for some people, but not necessarily everyone. So there was a lot of education before open enrollment that showed people how to think about their plans.\nspeaker 0: So that was the first piece. Then after open enrollment was done, we explained that that people in 2018 will no longer have to do the wellness assessment in individual action plan because in 19 it's sort of gold for all.\nspeaker 0: Everybody will be at the gold level. So that's the second part of the communication. And that happened in December. And there was a video and it's it's been widely watched. And so people people have that part. And then starting this in February and going forward, we will explain the new kinds of activities people can do to be\nspeaker 0: healthy. It's not tied to the level of plan you have, but it's a whole different way of doing wellness activities. So it's spread out over, over many months without trying to tell everybody all this and all of these things with all these acronyms all at the same time.\nspeaker 1: So how would our health care and health advisory staff know whether or not this new direction that we're going is going to bear fruit? Like, how am I going to how are you going to know whether that says cancel out?\nspeaker 1: The girl doesn't have to fill out the forms if he's still running and walking and exercising like he had been doing. How would.\nspeaker 0: It work? We're not trying to keep track of what every single person does. We're looking at things that we can offer in the workplace that help people be healthy. And so. They too, have different ways that people can participate.\nspeaker 0: But it doesn't. You're not going to have to fill out forms. It's not going to be tracking each person. It's going to be looking at what happens in larger workgroups and and measuring things like people's confidence that they understand the health plan, their confidence that they know, that they know things that they can do.\nspeaker 0: You know, just in the more of a survey, not not a whole lot of individual tracking.\nspeaker 1: Okay. So, Chuck, I never knew that I was there to start off, historically speaking and have started off with Ron's coverage, because I know.\nspeaker 0: You did when you do those actions in order to get to the higher levels, easier to complete those actions.\nspeaker 1: Yeah, but I know some people have been I go from day one, I'm not supposed to know that.\nspeaker 0: Oh.\nspeaker 1: Is it possible offensively or no.\nspeaker 0: Well, when we put the the healthy incentives program in place, depending on how you wanted to view it, you were either going to stay at the gold by doing those things, which is what the unions were explaining to their members.\nspeaker 0: Or technically it was a bronze plan and you earned your way up. The result was the same. If you did the individual action plan and you did the wellness assessment, then you got gold. So you could either say you stayed at that level because you did those things or you worked your way up from bronze because you\nspeaker 0: did those things.\nspeaker 1: All right.\nspeaker 0: Thank you. So I wouldn't have been here. Is it fair to say that the rationale here is that the executive branch believes that the use of the Affordable Care Network will provide better health outcomes at a lower cost than the use of the incentives program.\nspeaker 0: Is that am I hearing that, Cory? Because I didn't get a real rationale for why to essentially back away from incentives and not use incentives anymore. Council Member Baldacci That is correct. The the cost of health care, of not getting the right health care at the right time, either getting too much or too little or going to\nspeaker 0: a place that's more expensive without delivering better quality is a much bigger impact on our cost and actually our health outcomes. We certainly want to support people to do healthy things, to do what they can to control the the health situation that they have.\nspeaker 0: And we want to continue to support that. But the real truth is that the cost of services and the quality of services has a much bigger impact on on our budget and on how healthy our people are than than the lifestyle activities.\nspeaker 0: Thank you. And I guess if I may, I'm just going to follow up in Oakland Councilmember Lambert, the fiscal note that was handed out was pretty. It seems to show slight savings in 2018 and then a $500,000 approximate increase in 2019.\nspeaker 0: Can somebody put that in some context for me compared to what our costs have been and how much of a percentage increase that is and what's driving that?\nspeaker 2: Councilmember Bell Michelle, I'll try to answer those questions for you. The cost savings were as a result of the shift of employees from the PPO to the Select Plan and the anticipated costs. And it was based upon a projection of a 5% shift.\nspeaker 2: We actually had a 6% shift, but we wouldn't have known that at the time we were preparing the fiscal note. And with respect to gold for all in 2019, Mercer, who is a supports the county in its in its benefits administration had estimated that it would result in an additional cost of $600,000 to the overall benefits budget\nspeaker 2: , which is about $240 million per year. So the offset is obviously enjoyed by the region. And again, when I say the $600,000 increase in terms of the cost of the benefits, that is a projection that is really subject to really what the plan utilization will be by all members of the jail am.\nspeaker 2: I said so. So that number is just the best that Mercer can put on it, just anticipating the utilization of those that had been at the silver and bronze level previously.\nspeaker 0: So I'm doing some quick math up here. Never a good idea, but going with the numbers that you just shared, 240 million total health care costs to the county.\nspeaker 2: That's our budget. For health care. That isn't necessarily our costs to health care, that is our funding.\nspeaker 0: What has our costs been most recent number that we have?\nspeaker 2: I do not have the most recent number of the costs for you. Councilmember We can get that for you.\nspeaker 0: I'm trying to get at the curve. And so in the past, we've seen year over year health care increases in the double digits, 10% plus percent increases in health care. And and the county's been actively working to bend that curve downward while encouraging better health for employees for a while now.\nspeaker 0: And this seems to me to be the next evolution of that, but I'm just trying to get at how we're doing. So that would be really good to know coming forward.\nspeaker 2: We have we.\nspeaker 0: Can give you detailed numbers on that. We've actually gone from double digits to an average of around four and a half percent per year, which is which is doing better than the national average. Very good. So that that's all I really need.\nspeaker 0: I mean, that sort of basic information is helpful.\nspeaker 2: And then in our projections for the protected fund reserve, which is the bucket and where the money goes into or comes out of, depend upon whether or not our costs are higher or lower. But then, then our budget is looking pretty good now.\nspeaker 0: And I understand that one employee or one employee is dependent with a really particularly nasty form of cancer, can change the picture in, you know, in one year all by themselves because it can get very expensive. And that's expected that we will cover those sorts of, you know, situation.\nspeaker 0: So final question, is this a process and approvals? So this comes to us now here we are in January, but these plans were offered for open enrollment last year. What happens if we don't approve this?\nspeaker 2: Councilmember Bell Duchin that's a very fair question. And in terms of of that, I think when I sit here now, you know, in terms of having this discussion in January, one would have perhaps had two different agreements for each of the two changes, one in terms of age and the other one in terms of the health incentives\nspeaker 2: and the plan design. Because the gym I see as as you know, is authorized to bargain and implement plans for 2008. And with respect to these but at the time we were bargaining that we were hopeful that we would have gotten everything in front of you before open enrollment.\nspeaker 2: And we signed off on this in August or left my office in September and in a blink of time. Here it is in January. So so really what's here is in terms of looking at the continuation of these benefits in the 2019 benefit plan year.\nspeaker 2: Keeping in mind to Councilmember Bell duty that the GMAC as well as Total COB are continuing to be empowered to bargain changes the plan design. So in April we'll be looking at, you know, our projections for 2009 ten and beyond and making decisions of whether or not the plan designs that are before you now should be modified\nspeaker 2: . In addition, we're in total comp bargaining this year for successor benefits plan, which also may modify these as well. So the one thing about looking at this is looking at the continuation of these particular plans beyond 2008 ten and recognizing that logistics being what they are.\nspeaker 2: It wasn't a ideal situation.\nspeaker 0: Yeah, and I get that it's complicated and not neat. And the end of the year is challenging for a number of reasons. But I would just ask that we take the Council's approval of these things as something other than just a rubber stamp.\nspeaker 0: I would appreciate that is. Ed Yeah, because I still.\nspeaker 1: Didn't hear answer you as to what would happen if we're not okay yet today.\nspeaker 0: I think they would have to go back to the bargaining table and who knows what it would be a mess, I think, is what it would be.\nspeaker 2: Certainly, Madam Chair, I would I would think that for 2018, based upon the benefits agreement, we'd be fine. But we certainly would have to take a look at 2019.\nspeaker 0: That's the part that I'm not getting. So there's a benefits of like, did we and you'll have to forgive me for not knowing this, but did we pre-approve pre authorize the job of my C to decide within certain parameters or decide period and therefore whatever you guys come up with is okay, council approval notwithstanding.\nspeaker 2: Madam Chair, under the benefits agreement, it does talk about the fact that we are empowered to negotiate the modifications to insured benefits and implement those agreements, and that these for the age and any third plan option for 2018 and to be implemented on January 1st of 2018.\nspeaker 0: That was an agreement that previously was approved by the council that authorized what's happening here today.\nspeaker 2: Yes, ma'am. Answer for 2018 2019. Like I said, had we thought about this in terms of the timing of how long it take to get before you, it would have probably logistically been better for the executive branch as well as to generally say union says.\nspeaker 2: Well else to have had four agreements before you to recognize in the decisions we made for 2018 and to recognize in the negotiated agreements for 2019 as it was, we have them combined in 18 and 19 under single agreements for each of the provisions.\nspeaker 0: So I think the answer to my question, Councilmember Gossett, would be that it would be a bit logistically and administratively challenging to pull the two apart and do whatever we do for 2019. But let's hope that that's not necessary.\nspeaker 0: Okay. Councilmember Lamberts in waiting. Thank you. Just a couple of quick things. So you mentioned a minute ago that you were going to have healthy outcomes by work groups. And I think health is kind of an individualized thing.\nspeaker 0: So I am concerned about that. I don't want to let up on the idea that we all need to be doing things to take good care of ourselves. And I know that I'm that I think it was like five or six years ago when we first started the plan where you had information on how to do exercises\nspeaker 0: properly. I thought that was really good information. It's been years since people really were that specific with me on that. And I think, you know, I think that's a really good thing that we could share with our employees to encourage them to exercise, but also to do it correctly.\nspeaker 0: So I hope that we don't give away completely. I agree with the other council members I hate and making those forms that all the time. But it did I did learn a lot on some of the years, but not all of them.\nspeaker 0: I'm really thankful that you have other plans besides the PPO and the HMO so that people have an option if they choose not to have that particular kind of plan. We did a couple of years ago, and I think it was the group you just mentioned that Mercer, that they were looking at each of the hospitals and\nspeaker 0: a surgery in each hospital cost and made comparisons across the different hospitals and just have some numbers. This hospital has this survival rate and this hospital has that. And will we be seeing any more of that information? Because I think it's important for consumers to know how the hospitals are rating.\nspeaker 0: Councilmember Lambert, that work was actually done by the Washington Health Alliance. That's right. Which which was an organization that the county helped start on. They do have public reports and we can certainly give you information about where to find it.\nspeaker 0: It is public. It's public a by provider groups, individual, you know, clinics. So there's the community checkup report that shows the quality and outcomes for four clinics. There's a hospital report that shows variation and quality and cost. There's a new report that's going to be coming out that shows common things that the medical societies have agreed don't\nspeaker 0: need to be done and how much cost they add. So it's things like giving antibiotics for for bacterial infections and certain kinds of knee surgeries that don't don't really help people. And that report will be coming out soon.\nspeaker 0: And those are publicly posted on the Washington Health Alliance website. And we can certainly give you the the the website information for that. I think it also we should talk to public health about having a link from their website because when people go to look for health information, they might go to public health and be able to\nspeaker 0: see that their work. One of the things I mean one of the things I'm concerned about and it's happened several times in the last seven years, but as we change the pharmaceuticals, that they will say this is the formularies that we will pay for and these are the ones we want.\nspeaker 0: And then the next year it's different. And so if somebody is on a drug that's working for them and having to change and then change back, it's not good. And I think I've talked to somebody in your office about this.\nspeaker 0: So I think we need to be aware of that. And, you know, maybe having I think there's appeal process or something that your doctor can write and say, no, they don't want them changing to this drug. But I think trying to keep it a little bit more stable would be helpful.\nspeaker 0: And then in this plan, you know, we were supposed to be one of the early providers for the Affordable Care Act and all that. Is this connected to that at all, even tangentially? Councilmember Lambert Yes. This is one of the kind of health care innovations that was supported by the Affordable Care Act, because that the Affordable Care\nspeaker 0: Act comes with what's called the triple aim, which is to get better population health at lower cost with better patients. Satisfaction. And so the Accountable Health Network idea is very specifically designed to achieve those three things, so that that is very much in line with the Affordable Care Act.\nspeaker 0: So I know the employer that was in charge of getting us on that early adopters is no longer with us. So are we going to we're not going to be an early adopter. Are we still going to be a mid adopter?\nspeaker 0: Or when are we going to be adopting? And is this part of that or is this something separate? I'm not certain I understand your question because we certainly. It's mental health and physical health being combined. Yes, yes. Yes.\nspeaker 0: And so it's happening at two levels. We're doing it in the employee plan. But far more important, that's what's happening in the health department and community health, where they're working with the accountable, you know, accountable communities of health.\nspeaker 0: There's so many acronyms now. But yes, we are we are a mirror doctor in that there were two two counties that have a different kind of structure that were the the the test cases. And then we're in the next the the next wave and right on where we thought we were going to be.\nspeaker 0: Health Department can speak more eloquently to that piece. Sure. I think that would be good for us to know because other counties, because we were going to be an early adopter. Other counties were saying things like King County figured out first because they're bigger than us and we'll just go in the tailwind behind them.\nspeaker 0: And so other counties are calling me and saying, how far are you? And I don't really know. I mean, the affordable and the accountable communities of health. Yes. And when we're going to be fully adopted and and what have we learned along the way, which I went to one symposium from a county, and their lessons learned were\nspeaker 0: pretty interesting. So it might be good for us to know those things and maybe a briefing at some point on that would be interesting. Okay. I mean, it would be interesting to know what our experiences anyway. Right, for our own purposes.\nspeaker 0: So that'll be great. Before we close this out, I just want to make sure, Ms.. KOPPLIN, that you have an opportunity to say something if you choose. Don't feel pressured. But if you've done as the the the courtesy.\nspeaker 0: Thank you. Of of coming in being here this morning. So anything you'd like to add to what we've talked about today? Well, I would like to thank the council for their consideration on this. I'm really proud of our Joint Labor Management Insurance Committee.\nspeaker 0: I feel like this is an example of where labor and management can come together, sit down, work through problems, and come up with some really good outcomes. I'm also very proud of our employees because I will note that this year they did not increase the employer contribution for the insurance contributions.\nspeaker 0: They they took basically a 0% increase. And our our PSR is still doing very well, which I think speaks well to our employees and how well versed they are in health care and how this process has basically been a success in an arena where health care costs are often rising significantly.\nspeaker 0: So I wanted to acknowledge that and acknowledge my colleagues and, you know, encourage the council to really take the recommendations of the Joint Labor Management Insurance Committee seriously, because we work really hard together on this. And all of our decisions are very.\nspeaker 0: Well-thought out and discussed at length. So. All right. Well, thank you. Thank you all for for coming in, working us through this. This is before us for potential action today if we're prepared to move forward. It is proposed.\nspeaker 0: Audience Member 2017 2047572. That please come from a member. Madam Chair, I'm happy to put before you propose ordinance 2017 zero four I'm sorry 0475 with a do pass recommendation and the other comments or questions, Council Member McDermott, followed by Councilmember Dombroski.\nspeaker 2: Thank you, Ms. Palaszczuk. Your staff report analysis section ends with staff analysis is ongoing. Can you speak to what work is continuing?\nspeaker 0: Absolutely. Councilmember McDermott, that was based on the fiscal note that was provided to you today. There was addition information that council staff requested and that information has been provided.\nspeaker 2: Great.\nspeaker 0: Thank you. Thank you. Councilmember Tomasky, thank you.\nspeaker 3: Just right above it and returning to our opening colloquy here. The staff report notes that the human resource vision doesn't believe that there will be significant cost savings. Did there's a discontinuous discontinuation? Just I can't even speak of the healthy incentives program, the ending of the paperwork.\nspeaker 3: I just want to note that it may be hard to capture, but with 13,000 plus employees and their family members having to fill out those forms, that takes them away from their work. It takes them away from their families.\nspeaker 3: It's a small thing, but but it adds up. And I think there is an undocumented savings, but savings there. And so I just want to commend you. You articulated it so well when you said, you know, if you're stuck down on that gold or bronze or silver level, the costs are higher to get in to see a\nspeaker 3: professional. It's a barrier there. People may not go. So I think your shift in thinking on that is great. I really want to endorse it from a practicality perspective, from a reasonable perspective, and from a health care perspective, I think you're headed in the right direction.\nspeaker 3: So thank you for your work on that. And Denise, thank you for our work working so hard on behalf of our representative employees. I really agree with everything you said that we have great partners there.\nspeaker 0: Councilmember Dunn, followed by Councilmember Jamaica.\nspeaker 2: Anyone from the insurance carrier here today? No, sir. Okay. So just labor and staff. Okay. I appreciate it and I appreciate all your work. So I'm not going to wrap you into this. But I called the local or the government direct relations director four times last year.\nspeaker 2: Four times they were going to call back. Yeah. Which is kind of reflective of the level of service I think we get from this insurance carrier. So on my desk is a ordinance which I've shown to several members up here that would go in a different direction with our insurance carrier as a county.\nspeaker 2: So at some point I'd sure like to have a conversation with somebody from some regions. They want to give me a call back. I'd be grateful. If not, then I'll just move the legislation. Council Member Doug, we'll follow up with your office to get more information on that so we can take your time and I'd be grateful\nspeaker 2: . Thank you. I think it's important that people are responsive and especially the tens of millions of dollars. And I also have other concerns which I can get into later. But thank you. I'm grateful. Thanks for being here.\nspeaker 0: Councilmember Lambert would like to comment on that topic as he goes is going to say that are on the phone with me and and they were answering my calls but yeah they are so anyway. Okay, maybe you got the wrong phone number.\nspeaker 0: Councilmember McDermott.\nspeaker 2: Thank you, Madam Chair. You've certainly heard some clearly interest, frustration, maybe even on occasion disillusionment with the health incentives program. And I'm not I think there is some value in the concept, but as I expressed to several people in the executive branch over the last year, I have some grave concerns about the current implementation of that program\nspeaker 2: in mind was much more specific to record keeping and whether we were seen if somebody could enter data in the right timeframe versus if they're actually engaging in activities to improve their health. In my conversations with the executive branch, I learned that a bigger concern driving some of their work to reform it is that healthy incentives had\nspeaker 2: those who were reaching gold didn't reflect our in our workforce in its entirety, that there were ESG implications in who was reaching gold and who wasn't in that concert. That concerns me even more so than the frustration of filling out forms.\nspeaker 2: I think there's actually value, as frustrated as it may be in taking that health and assessment. But I want one that. Helps our employees achieve their health goals and have better outcomes. So I'm not looking to chuck such a program, but I need one that actually is helping our entire workforce achieve better outcomes, not just segments of\nspeaker 2: our workforce. I also need one that is outcome driven, not input driven. Do you check the rate box to input the data on the right time frame? Thank you.\nspeaker 0: Well, just to wrap it up and then we'll we'll be ready to take action. I think that's what I heard is the direction and the idea of looking at the outcomes and looking at them in a population level as opposed to an individual level.\nspeaker 0: I heard that pretty clearly from the presentation is the hope for this new direction? And Councilmember Lambert reminded me, but I think it's fair to point out, as much as we've all come to hate the poor, unloved incentives program, it won awards back in the day when it was first put in place.\nspeaker 0: And it did a lot of good, I have to tell you. It did. It was not a bad thing for me to have personally to have a reminder at the beginning of each year to be thinking about my health.\nspeaker 0: It made me go get my checkups. It made it made me do things that, while they are indeed a hassle, are probably important to to health. And look at me here. I'm dripping. I'm a drippy, feverish mess. And I you know, if we had had our incentives program in place this year, maybe I wouldn't be.\nspeaker 0: So let's just put it that way anyway. Okay. Yeah, I think we're done with comments and we're ready to move to a vote rolls marker to call the roll. Please repeat before you do. Do we not have a motion?\nspeaker 0: I might know people in motion. We do? Yes. You need one? Yes. Another? Not that out of it. I recall the motion. Okay. Please go ahead and call the wrong. Thank you, Madam Chair. Councilmember Dombrowski. Councilmember Dunn, I.\nspeaker 0: Councilmember Gossett.\nspeaker 1: I.\nspeaker 0: Council member. Coles. Council member lambert, I. Councilmember McDermott All right. Councilmember up the grove. Councilmember one right there. Madam Chair. Madam Chair. On the vote, a seminar is no nos. Council members. Cole Wilson one mike there excused by your vote you have moved proposed ordinance number 2017 0475 with a do pass recommendation.\nspeaker 0: I think we really. No not at this one quite a bit. So we'll put it on consent if I don't see any objections. Not seeing any objections, it will be on consent. Thank you very much, everyone. All right.\nspeaker 0: We have two final items. Maybe we can get through them relatively expeditiously. Both are about solid waste. So I'm not I'm going to skip my introduction and just call on our staff. Mary and John and Terre Rose to walk us through both of these items, and we'll take them up one at a time.\nItem 2018-0062 transcripts:\nspeaker 0: We have two final items. Maybe we can get through them relatively expeditiously. Both are about solid waste. So I'm not I'm going to skip my introduction and just call on our staff. Mary and John and Terre Rose to walk us through both of these items, and we'll take them up one at a time.\nspeaker 0: Thank you. Thank you, Madam Chair, and members of the committee. For the record, I'm here, Rose. And with me is Mary Bergeron of Council Staff Proposed Motion 2018 0062 would acknowledge receipt of a report on garbage collection in unincorporated areas in response to a budget proviso requirement.\nspeaker 0: The materials for this item begin on page 141 of your packet. The Council adopted a budget proviso as a part of the 2017 2018 biennial budget requiring the executive to conduct a public outreach process, prepare additional analysis on recycling rates, and provide a report to council prior to advancing any proposal to reduce garbage collection frequency from every\nspeaker 0: week to every other week in the unincorporated area, the proviso is written such that if garbage collection frequency was not planned to be reduced, the report to council could say this and no further analysis or reporting was required.\nspeaker 0: On November 15th, 2017, the executive transmitted a letter to the Council stating that reduced garbage collection will not be pursued from every week to every other week in unincorporated areas of the county, from the state through the end of 2018.\nspeaker 0: And the executive's letter, as transmitted, satisfies the terms of the proviso the Council will have the opportunity to set longer term policy direction regarding garbage collection and recycling rates within the context of the Comprehensive Solid Waste Management Plan, which is anticipated to be transmitted later this year.\nspeaker 0: That concludes my staff report, and I'm happy to take any questions. Okay. I'm sorry. Could you say your last sentence again? There's a little. Sure, sure. I was just mentioning that the Council will have a property opportunity to set longer term policy direction concerning garbage collection and recycling rates within the context of the comprehensive Solid Waste Management\nspeaker 0: Plan, which is anticipated this year. Okay. So this is just I need to get up to where we're at. We are item eight Web page area of 141 141.\nspeaker 4: And the motion is on page 147.\nspeaker 0: That's what I'm looking for. It's a so the motion is to acknowledge receipt of the report. That's what we are being asked to do today. Is there comments or questions about this item? Councilmember two.\nspeaker 2: Thanks. So the report that has come back to us that we are accepting now says that it won't we won't go to every other week garbage collection for the duration of what period.\nspeaker 0: The letter filed with the clerk is from the date. So November 15, 2017 through the end of 2018.\nspeaker 2: Through the end of the year. Okay. But after that, and there hasn't been any commitment one way or the other in terms of from solid waste on the issue. Correct. Okay. Just want to make sure we have all the facts.\nspeaker 2: I have some comments. We're just accepting report and I know some comments.\nspeaker 0: Are just or something you report. So if you have comments, now would be the time.\nspeaker 2: Okay. So just real quick. So again, this accepts the proviso I sponsored and it essentially requires the division to either do significant outreach if they are going to go from one week to two weeks or to delay the policy until after the current by any.\nspeaker 2: And so the division is acknowledged and I think it's great that they're not pursuing every other week garbage collection policy through the current year. But I just want my colleagues to know that when I was first elected to the council, illegal dumping was a huge issue in my district and it continues to be a huge issue in\nspeaker 2: my district. Rates at the transfer stations, combined with growth, have just created a situation where all the people are going out to the unincorporated areas and dumping their trash or washers and dryers or even cars in some extreme situations.\nspeaker 2: So as part of that response, I sponsored legislation that created a voucher program to help low income residents clear dumping grounds. And in the first year, over half the vouchers had been awarded, which just goes to show the extent of the problem.\nspeaker 2: Also, I represent a lot of neighborhoods that are, you know, very unhappy with frequent odors coming from the facilities that we have out there. Of course, in my district, the landfill material composting is out there. There's a lot of odors coming from those facilities as well.\nspeaker 2: And so switching every other we garbage collection is probably going to drive up the collection of work organics which are sent to my district for processing and increase the smell. And with the current rate that the folks have been contacting my office about odors, I'm not comfortable with a every other week policy change going forward until there's\nspeaker 2: a plan to really address the odor concerns. You have no idea how many thousands of emails I get because of the odor related problems down there in that quadrant of the county. So I had a chance to survey my constituents with my newsletter here just last year, and I wanted to get a sense of how they felt\nspeaker 2: about this issue. And and well over half of the residents surveyed said they would be unsatisfied or very unsatisfied with this policy change. And you can understand why. I mean, it's you know, that, you know, you've got situations like, for example, you've got all kinds of critters out there in the unincorporated areas, especially rural King County.\nspeaker 2: And if you're leaving your garbage out for two weeks as opposed to one week, there's a greater opportunity for critters to get in there. There's smell issue, there's rodents and sanitation related issues. And and it's just it's just one of these things that and it's also, you know, even if the containers increased slightly, you still got an\nspeaker 2: issue. You know, we're going to charge them the same amount for the same, but we can charge them the same amount for half the volume being reduced from their house for or from their home. Right, if they're only coming every other week.\nspeaker 2: So those problems that I really have and so I just don't think this is a place to experiment and we'll be very dogged in pursuing that. Continuing on beyond the by any means. I know all the waste understands and appreciates.\nspeaker 2: I just wanted to get that information out. Thank you.\nspeaker 0: Yes.\nspeaker 2: And I would urge we accept the report. Okay.\nspeaker 0: You can simply thank you. Well, I just want to say I agree completely with the previous speaker that we are per capita. We have lots and lots of families out there. And because there's more families with more people in the house that you are going to have more garbage than.\nspeaker 0: You know, averaging a, you know, one or two person household. And as he said, garbage shows up elsewhere. And it has gotten, in some cases, so extreme that we've had to hire a helicopter to come in and bring large items out that have shown up in all kinds of crazy places, which is not safe for other people\nspeaker 0: and just makes it an eyesore. So I think, Councilmember, one of the things that your people need to know that the current garbage plan would take the at the height of garbage piled councilmember, the height of garbage piled up now can go up to 780 feet above sea level.\nspeaker 0: But the proposal is it would go to 800 and I think 20 feet above sea level. So that they're complaining now just wait unless we change our plan. But the plans before us, we'll have an opportunity to talk about that.\nspeaker 0: The other thing is that because of the garbage issues, we do have beer cans available and I think it's $4.99 more a month if you want a beer, garbage can. So for those people out there that are having problems with the bears, you can under our contract ask for a beer, garbage can.\nspeaker 0: Thank you. Thank you. All right. If there's no additional commentary, I would accept a motion to approve motion number 2018 0062 of the do pass recommendation.\nspeaker 2: Gladly. Madam Chair, would we propose motion number 20180062 with a do pass recommendation? But we would also ask that it be removed from the consent agenda very well.\nspeaker 0: Any other comments or questions? I'll ask her to please call the roll. Thank you, Madam Chair. Councilmember Dombrowski. Councilmember Dunn.\nspeaker 2: All right.\nspeaker 0: Councilmember Gossett.\nspeaker 1: I.\nspeaker 0: Council Member Cole Wells. Councilmember Lambert. Councilmember McDermott. All right. Councilmember Staff members. Councilmember one right there. Madam Chair, I. Madam Chair, the vote of Seven Eyes, No No's, council members Cole Wilson and one. Right. They were excused. All right.\nspeaker 0: By your vote. You have. We have passed proposed motion number 2018 0062 of the do pass recommendation. That is to accept the report and it will not be able to consent calendar. So I'll move on to the last item on our agenda today, which is proposed motion number 2017 0505.\nItem 2017-0505 transcripts:\nspeaker 0: By your vote. You have. We have passed proposed motion number 2018 0062 of the do pass recommendation. That is to accept the report and it will not be able to consent calendar. So I'll move on to the last item on our agenda today, which is proposed motion number 2017 0505.\nspeaker 0: It's another solid waste motion, this one accepting a report regarding system tonnage because the county Cedar Hills Land Regional Landfill is nearing its capacity. As we all have discussed here many times, it's important to carefully monitor the tonnage that we're disposing and study options for long term waste disposal.\nspeaker 0: As part of that effort, the Council adopted a proviso, you will recall, in the 2017 2018 budget that required two reports from the Solid Waste Division. The first, which we will hear about today, is on 2017 system tonnage.\nspeaker 0: The second is due in June on long term options for the landfill. So once again, we have with us Mary Bergeron and Tara Rose to help us with a briefing.\nspeaker 4: Thank you, Madam Chair. Mary Bergen on from the council staff, for the record, and you have introduced the council's rationale for this proviso. As you noted, this system tonnage report from 2017 is the first of two reports that will be required on this topic.\nspeaker 4: And if you will move ahead to page 155 in your packet, I will share the results of the analysis that the division did to prepare this report. What they did because this report was due on December 1st of last year, this report provides actual tonnage figures from January through August and then estimates for the remainder of the\nspeaker 4: year. And when you see the second report, which is coming in June, you'll see actuals for the entire year. I will note first that in preparing to do this report, the Solid Waste Division had prepared tonnage estimates through the year 2036 and they had presented those to you back.\nspeaker 4: At the same time, you adopted these provisos when you were adopting the solid waste rate. These and if you look on page 156, you can see a table that outlines what the projections were for each year. And if you look at the shaded row 2017, you'll see that the division was estimating about 851,000 tons last year.\nspeaker 4: They had estimated, if you look, a little bit of a decrease from the year before because of the reopening of the city of Seattle's North Transfer Station and Recycling Center, which they estimated would take some of the load off the county stations.\nspeaker 4: If you then jump to the bottom of page 157 and look at the actual results that the solid waste division has transmitted, you will note that tonnage, in fact increased 10% above those projections. So 932,000 tons rather than the 851,000 that were projected.\nspeaker 4: This has implications for the landfill capacity estimates that this additional 81,000 tons will shorten the landfill life by about a month. And moving on to the next page. Also has operational impacts because this additional tonnage was about 230 more tonnes of waste per day that were transported to the landfill than had been estimated.\nspeaker 4: That resulted in about 3700 additional roundtrips between the transfer stations and the landfills last year. The division has noted that they have been addressing these operational impacts through overtime to date, but may be coming back to the Council for additional appropriation or possibly even FTE authority.\nspeaker 4: And finally, I'll note that this additional tonnage resulted in about $11 million in additional revenue above and beyond what had been forecasted for last year. And the division notes in that case that they recognized they don't have appropriation authority to use that additional revenue and expect they may be coming to the Council through one of the supplemental\nspeaker 4: budget ordinances. I will close by noting that the additional unanticipated tonnage for last year will be addressed in more detail. In the second of the two provides a report to the one that will be coming to in June that will also include more information about the landfill.\nspeaker 4: And I think this does provide a good segue way for the Council to your upcoming discussions and deliberations on the Comprehensive Solid Waste Management Plan, a draft plan of which was released last week. And the executive's recommended plan is anticipated to come to the council probably mid this year.\nspeaker 4: So this kind of sets, I think, the ground for what you will be looking at. And with that, Madam Chair, that concludes my staff report.\nspeaker 0: Okay. Thank you very much. Councilmember Lambert has a question. Thank you. So help me on these charts. So special waste is going to go from 1500 to 2500 in approximately 20 years. What is changing that? That is going to be so substantially the same that that's going to happen because it increased population.\nspeaker 4: So I think we've got staff from the division here and I think they might be best to answer that question. I had been focusing mostly on the total column and focusing mostly on 2017 things. That was the topic of this report.\nspeaker 0: And talk about that when we meet next. And the same thing in the next column, going from 1300 and 13,500 in yard waste down to 16. But, you know, lots of people are doing rock gardens and other kinds of gardens so that there aren't as many yard ways.\nspeaker 0: So we'll add that to our list next time we talk. All right. Thank you. Okay. So one of thing, please. Anything is I. I just saw and something that was written that that a clean wood is no longer going to be accepted at the house and transfer station.\nspeaker 0: And I didn't know that was happening until it apparently happened. So I'd like to talk about how that happened, and I'm not really happy about it. So thank you. Okay. I don't know if we do have executive staff here.\nspeaker 0: I don't know if they want to come forward and say a few words before we go ahead and accept the report. But yeah, only if you want to get in and you're not required.\nspeaker 2: If you have a question that we'd like to.\nspeaker 0: Hear enough. I do have one question, and that is when this projection comes in a significant marginal but significant amount higher, this actual annual tonnage comes in at this level higher. And it says that that would that would mean the capacity of the Cedar Hills landfill be reached about one month sooner.\nspeaker 0: Does that assume that the rest of the projection comes in as previously projected, or do we now reset the rest of the projection and say even if things increase at a rate similar to what we assumed before, but now we're up to 932, we're starting with a different base.\nspeaker 0: Is that how you get to one month? I mean, I know that this is we're prejudging the question of the next report, but it seems to me that that seems odd that that much of a change to the curve would result in such a small impact to the life of the landfill.\nspeaker 0: The Please Make Morehead King County Solid Waste Strategy Communications and Performance Manager. This provider was very specifically focused on 2017.\nspeaker 4: And so the one month change in the landfill.\nspeaker 0: Life is based on the change in tonnage for 719 2017 only. And it's the second proviso that speaks to the broader change in the overall projection. Manage and so forth. Okay. Once again before us, today is just the motion to accept the report, and that's motion number 2017 0505.\nspeaker 0: If there's no further discussion, I would entertain a motion. Thank you, Madam Chair. I'd like to propose a proposed motion 2017 0505 with a do pass recommendation. All right, it's been moved. Any comments or questions saying nonetheless the clerk to please call the roll.\nspeaker 0: Thank you, Madam Chair. Councilmember Dombrowski. Councilmember Dunn. Councilmember Gossett. Council Member. Cole Wells. Council Member. Member. High Council Member McDermott. High Council member of the Grove Council. Member of one right bar. Madam Chair. Hi, Madam Chair. The vote is six eyes.\nspeaker 0: No nos. All right, by your vote, we have approved proposed motion number 2017, dash 0505 with a do pass recommendation. This one also, I think should be on the regular course along with the other one. And that brings us to the end of our agenda today.\nspeaker 0: Our next meeting will be Wednesday, February 7th, when we anticipate continuing our work on the King County Code, changes to make it before gender neutral, and receiving a briefing about the Hearst Supreme Court decision and for cultures budget and.\n"

In [18]:
def file_inference_mb(question, material):
    prompt = prompt_no_input.format_map({"instruction": material + "\n%s"%question})
    output = respond(prompt=prompt)
    print(output)

In [19]:
file_inference_mb(question, material)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Here is a summary of the meeting in two sentences:

The meeting focused on discussing various county health care and solid waste related items. The council heard presentations from the Joint Labor Management Insurance Committee and the Solid Waste Division regarding changes to the health insurance plan and waste management policies. The council approved changes to the health insurance plan that will provide better outcomes for employees while saving costs. However, some were unhappy with the health incentives program and wanted to replace it with a new accountable health network. The council also discussed changes to the solid waste plan that will reduce collection from every week to every other week to save money. While the council accepted the reports, some were concerned about the small but significant increase in tonnage projections that will shorten the life of the landfill by about a month.


### File

In [17]:
def file_inference_mb(question, in_file, out_file):
    with open(in_file, "r") as f:
        data = json.load(f)
    with open(out_file, "w") as f:
        for meeting in tqdm(data):
            with torch.no_grad():
                sample_result = {}
                material = meeting["source"]
                target = meeting["summary"]
                prompt = prompt_no_input.format_map({"instruction": material + "\n%s"%question})
                output = respond(prompt=prompt)
                sample_result["target"] = re.sub(r'\n', '', target)
                sample_result["prediction"] = re.sub(r'\n', '', output)
                json.dump(sample_result, f)
                f.write('\n')

            del sample_result, material, target, prompt, output
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

In [18]:
in_file = "../../MeetingBank/test_text.json"
out_file = "./output/output_MB_text.jsonl"

In [19]:
file_inference_mb(question, in_file, out_file)

  0%|          | 0/199 [00:00<?, ?it/s]

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
100%|██████████| 199/199 [4:52:49<00:00, 88.29s/it]   
